<a href="https://colab.research.google.com/github/ahluwalij/BigThink_AI_Fall_2022/blob/main/BigTh!nk_AI_Project_1_Fall_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Introduction

**Welcome!**

Hey, there! Welcome to your first project at BigTh!nk AI. You're going to build your own AI! There's some reading to do along the way, but it's going to be fun and worth it at the end!

**Making a Copy**

**Log into this file using your UMD email account!** Only a copy of this file should be modified, *not* the "BigTh!nk AI Project 1 Fall 2020.ipynb" file itself.

1. Go to "File" at the top left.
2. Select "Save a copy in Drive".
3. Open that copy, it should be in a new tab with the title "Copy of BigTh!nk AI Project 1 Fall 2020.ipynb". Rename this to anything you want! Keep the .ipynb extension, though.
4. Share your copy with your group members using "Share" at the top right.

**Asking For Help**

During this whole thing, if you need any help at all, click "Ask for help" at the bottom of your Zoom Breakout Room or ping us on the BigTh!nk Discord server's #project-team channel with @AI Lead, @Python Lead, @DS Lead, or @President. We'll be right with you.

**Runtime Instructions**

Remember, this environment divides all your code into cells.

1. Connect to a Google server, top right. Wait for it to say "Connected".
2. Go to "Runtime", top left. Select "Change runtime type" and set it to GPU.
3. Click the `[ ]` on the left of each code cell to run it. Or just hit "Runtime" and select "Restart and run all" if you need.

**Note:** If you run a cell, but modify something in a cell above it, then to avoid errors, you have to rerun every cell from and including the cell you modified.

**Note:** If your syntax and tensor dimensions are correct but you're still getting an error, go to "Runtime" at the top left of this Colab and select "Restart and run all".

**Google Drive Instructions**

This will help us save your work.

1. In the cell below, enter your group number in the `GROUP_NUMBER` variable, and run it.
2. You will see a link starting with `https://accounts.google.com`. Click it. 
3. Log in using your UMD email account (on which you have the BigTh!nk AI shared drive). 
4. Click "Allow".
5. Copy the displayed code into the box here, under the code cell, saying `Enter your authorization code:`, and press Enter.

In [ ]:
# Write your group's number below!
GROUP_NUMBER = 

from google.colab import drive # ADD TEXT ABOVE
drive.mount('/content/drive')

Here are some libraries you'll see very often in machine learning and deep learning. Remember, importing TensorFlow implicitly imports Keras! Go ahead and run this cell below with the imports. It'll take a minute.

In [ ]:
import tensorflow as tf # our favorite AI framework!
# Note that keras is imported by default, so we'll often call tf.keras.<command>

import math
import numpy as np # the fundamental building block of ML: arrays!
import matplotlib.pyplot as plt # this will help us plot and visualize our data
import logging
import seaborn as sns # this will help us in understanding performance metrics towards the end

!pip install -U tensorflow_datasets # we're getting the repository of datasets that this project's dataset belongs to

import tensorflow_datasets as tfds 
tfds.disable_progress_bar()

logger = tf.get_logger()
logger.setLevel(logging.ERROR)

     |████████████████████████████████| 3.6MB 2.8MB/s 
  Found existing installation: tensorflow-datasets 4.0.1
    Uninstalling tensorflow-datasets-4.0.1:
      Successfully uninstalled tensorflow-datasets-4.0.1


# 2. The Problem

**Issue**

It's 2021 and there's a new malaria outbreak! People in Yew Nork City are shivering in fever and chills! It's up to you, the AI engineers at BigTh!nk BioTech (established after the famous coronavirus pandemic of 2020) to speed up diagnoses.

With patients storming hospitals, we need a way to make testing incredibly rapid and accurate - and the current methods are far too slow. We need you, and we need the power of AI.

**Task**

Your job is to make a model that can learn from previous images to identify a malaria-infected cell. By the end of this project, given an image of a cell, your model's prediction must be one of two outputs:

0: Uninfected

1: Parasitized

**Supervised Learning**

Remember, a supervised learning AI will look at many input examples (images), look at what the output actually was (labels) for those input examples, and then try to figure out what the link is. 

Once it learns that, it will try to predict the output of an unseen, previously unknown input datapoint (a new image).


# 3. The Raw Data

The hospitals of Yew Nork City have compiled 27,558 cell images from thin blood smear slide images. If you're not sure what that is, don't worry! It's just a way to take pictures of microscopic cells. These pictures are what you'll train your algorithm on to identify which ones are parasitized.

Go ahead and run these code cells below to load the dataset and divide it into a training dataset, `train_ds`, and a test dataset, `test_ds`.

Here's our split:

`train_ds`: 70% | `test_ds`: 30%

In [ ]:
ds, info = tfds.load('malaria', split = 'train', shuffle_files = True, with_info = True)

train_ds, test_ds = tfds.load(
  'malaria',
  split = ['train[:70%]', 'train[70%:]'],
  shuffle_files = True, as_supervised = True,
)

Shuffling and writing examples to /root/tensorflow_datasets/malaria/1.0.0.incompleteRYUDHZ/malaria-train.tfrecord
Dataset malaria downloaded and prepared to /root/tensorflow_datasets/malaria/1.0.0. Subsequent calls will reuse this data.


In [ ]:
print(train_ds)

<_OptionsDataset shapes: ((None, None, 3), ()), types: (tf.uint8, tf.int64)>


Run this cell to see exactly how many images are in each of our two datasets!

In [ ]:
NUM_TRAIN_IMAGES = tf.data.experimental.cardinality(train_ds).numpy()
print("Number of training images: " + str(NUM_TRAIN_IMAGES))

NUM_TEST_IMAGES = tf.data.experimental.cardinality(test_ds).numpy()
print("Number of testing images: " + str(NUM_TEST_IMAGES))

Running this cell below will show you some examples of raw, unprocessed images of our patients' cells and their labels:

In [ ]:
vis = tfds.visualization.show_examples(ds, info)

Take a look at the labels of the images above. The labels in this raw dataset are unintuitive. Currently, the labels are:

0: parasitized

1: uninfected

"*A tragedy! Not intuitive at all! Where is the concern?*" - unknown source

We're going to invert these in **4. Processing the Data**, so that the labels will *later* be:

0: uninfected

1: parasitized



If you take yet another look at the pictures above, you might notice the images aren't evenly sized.

Running the cell below can show you what we mean by uneven image size.

In [ ]:
for image, label in train_ds.take(3):
    print("Image size: ", image.numpy().shape)
    print("Label: ", label.numpy())

If the output above is something like:

`Image size: (140, 120, 3)`

it just means the image is 140 x 120 pixels, and the 3 tells you how many color channels are in the image. There are 3 here, which are red, green, and blue (RGB). Pixel stuff.

# 4. Processing the Data

If you're supposed to build an AI to learn well from these images, you need some cleaner data! It makes it easier on whatever model you build to learn. Don't worry, we're handling this part for you! Just go ahead and run this cell below.

We're doing two things:

* Standardizing image sizes to 200 x 200 pixels
* Inverting labels to be 0 for uninfected, 1 for parasitized

This cell defines a couple of standard constants we want, as well as 3 functions we've written to clean the data. It'll call those functions on `train_ds` and `test_ds` to create `clean_train_ds` and `clean_test_ds`.

In [ ]:
BATCH_SIZE = 32
IMAGE_SIZE = [200, 200]

def convert(image, label): # this will be called by the next function, pad
  image = tf.image.convert_image_dtype(image, tf.float32)
  return image, label

def pad(image,label): # resizing each image to 200 x 200
  image,label = convert(image, label)
  image = tf.image.resize_with_crop_or_pad(image, 200, 200)
  return image, label

def invert_labels(image, label): # switching the 0 and 1 around, as mentioned
  return image, 1 - label

clean_train_ds = (
    train_ds
    .map(pad)
    .map(invert_labels)
)

clean_test_ds = (
    test_ds
    .map(pad)
    .map(invert_labels)
) 

# 5. The Clean Data

Run this cell below and look how clean these examples are! All the cell pictures have been padded and fit into images that are 200 x 200 pixels. These are ready to be read by your model.

As you might have noticed in the cell above, the names of the clean training and testing datasets are `clean_train_ds` and `clean_test_ds`. We're going to use Matplotlib to visualize our changes to the data.

In [ ]:
image_batch, label_batch = next(iter(clean_train_ds.batch(BATCH_SIZE)))

def show_batch(image_batch, label_batch):
    plt.figure(figsize = (10, 10))
    for n in range(25):
        ax = plt.subplot(5, 5, n+1)
        plt.imshow(image_batch[n])
        if label_batch[n]:
            plt.title("parasitized (1) ")
        else:
            plt.title("uninfected (0) ")
        plt.axis("off")
show_batch(image_batch.numpy(), label_batch.numpy())

Looks like the labels are indeed fixed! 0 now means uninfected, and 1 now means parasitized.

Don't worry about this little cell below, just run it. We're just getting the data in batches to be sent to the model for training and testing.

In [ ]:
clean_train_ds = clean_train_ds.repeat().shuffle(NUM_TRAIN_IMAGES).batch(BATCH_SIZE)
clean_test_ds = clean_test_ds.batch(BATCH_SIZE)

# 6. The Model

**Your Job**

Phew! We're finally here. Your job is to create and optimize a CNN model to identify malaria in a cell. As a re-statement: the model should produce 0 on an uninfected cell image and 1 on a parasitized cell image.

**Convolutional Neural Networks (CNNs)**

To detect malaria, we need some way to *see* the cells! In computer vision, this is commonly achieved through CNNs. A flashlight in the dark, seeing part by part until it pieces the picture together.

We're writing the convolutional part of the network. **Don't change it!** 

Write your part of the model under `# BUILD YOUR PART HERE`, and run the cell.
`model.summary()` will give you a good idea of your finished network.

**Output**

**Do not modify the output layer.** It needs to be 1 neuron with a sigmoid activation function, because this problem is a binary classification problem (only two outputs, 0 or 1). Don't worry about rounding anything, Keras does all that internally.

**Hyperparameters**

Remember, *parameters* are what the model adjusts by itself. *Hyperparameters* are what *you* can build and adjust in the model to maximize performance. In this project, you're free to change any or all of the following:



*   Depth of neural network (number of dense layers except output)
*   Width of neural network (number of neurons per dense layer except output)
*   Activation function of each dense layer (except output)

You *can* also change the following, but we'll get to these two in **7. Compiling the Model**.

*   Optimization algorithm
*   Learning rate

Hint: you might find this useful. [Keras Sequential Model](https://keras.io/guides/sequential_model/). Remember to prefix layers with `tf.keras.`!

Go save Yew Nork City!

No pressure.

In [ ]:
model = tf.keras.Sequential([
                             
    # CNN: this is the convolutional part of the neural network, we got this

    tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = tf.nn.relu, input_shape = (200, 200, 3)),
    tf.keras.layers.MaxPooling2D((2, 2), strides = 2),

    tf.keras.layers.Conv2D(32, (3, 3), padding = 'same', activation = tf.nn.relu),
    tf.keras.layers.MaxPooling2D((2, 2), strides = 2),

    # BUILD YOUR PART HERE:



    # AND YOU'RE DONE! Don't modify this output layer below.

    tf.keras.layers.Dense(1, activation = tf.nn.sigmoid)    

])

model.summary() # this is going to print a quick little summary of our model

# 7. Compiling the Model

There are three parts to the `model.compile()` function: `optimizer`, `loss`, and `metrics`. While you're only supposed to change the `optimizer` (and learning rate) in this project, we're going to explain them all.

**Optimizer**

You are free to set an optimizer you think works best! You can find a list of [Keras optimizers here](https://keras.io/api/optimizers/) if you scroll down to the "Available Optimizers" heading.

Remember, setting an optimizer can be done either by passing a string, like 

`optimizer = 'RMSProp'`

or by calling a function from `tf.keras`. For example, 

`optimizer = tf.keras.optimizers.RMSProp()`. 

We recommend the latter, since you can set `learning_rate` as a parameter of the function, like 

`optimizer = tf.keras.optimizers.RMSProp(learning_rate = 0.1)`

(Hint: don't just settle for RMSProp!)

**Note:** If you do not explicitly set a `learning_rate` value, it'll go by the default value listed in the documentation. Default `learning_rate` values can be found on that optimizer's documentation ([click here](https://keras.io/api/optimizers/)).

**Loss Function**

We've set the model to use the `BinaryCrossentropy()` loss function, commonly used in binary classifiers. The model adjusts its parameters (weights and biases) to minimize this loss. **Don't change this!**

Remember, the lower the loss, the better!

**Metrics**

Loss is fine to understand if the model is learning. But in the real world, a common way to test is called the **confusion matrix**. We'll get to this in **9. Performance on Test Data**, but basically, it has four metrics we calculate, given by the bullets below. And you can check this link for all possible [performance metrics on Keras](https://keras.io/api/metrics/). **Don't modify this part of the function!**

Here are the metrics that will help us measure your models' performance.

*   `TruePositives` (guess correct, guessed 1)
*   `TrueNegatives` (guess correct, guessed 0)
*   `FalsePositives` (guess wrong, guessed 1)
*   `FalseNegatives` (guess wrong, guessed 0)

In [ ]:
model.compile(# set the optimizer of your choice below! Learning rate too, if you want.
              optimizer = ,

              loss = tf.keras.losses.BinaryCrossentropy(),
              metrics = [tf.keras.metrics.TruePositives(), tf.keras.metrics.TrueNegatives(), tf.keras.metrics.FalsePositives(), tf.keras.metrics.FalseNegatives()])

# 8. Training

**Epochs and Batches**

When you read a chapter in a book, that's basically a *batch*. When you read the book end to end once, that's basically an *epoch*. 

The same way, when a model reads a group of (in this case) 32 examples, it's a batch. That's `BATCH_SIZE = 32`, which we defined at the start of **4. Processing the Data**. When a model reads through all of the training examples once, it's an epoch. That's `NUMBER_OF_EPOCHS = 5` here, meaning 5 epochs. **Don't change `NUMBER_OF_EPOCHS`!**

`steps_per_epoch` is just a calculation of how many batches there are per epoch.

**Beginning Training**

The `model.fit()` function from Keras begins to train the model and tweak its parameters (weights and biases).
Run this cell below to begin the actual training process for the model. You will see a progress bar, followed by the `loss` value and the number of `true_positives`, `true_negatives`, `false_negatives`, and `false_positives` in that epoch.

Running the `model.fit()` function once can take around 3-4 minutes on the GPU setting.

In [ ]:
NUMBER_OF_EPOCHS = 5

model.fit(
    clean_train_ds, 
    epochs = NUMBER_OF_EPOCHS, 
    steps_per_epoch = math.ceil(NUM_TRAIN_IMAGES / BATCH_SIZE))

Great! You were able to train your model! 

**Saving the Model**

After 5 epochs, your model has tuned its weights and biases in a certain configuration. To save that model in the BigTh!nk AI shared drive, just run the cell below. Don't edit it.

In [ ]:
model.save("/content/drive/Shared drives/BigTh!nk AI/Models Proj1 F2020/%s.h5" % GROUP_NUMBER)

# 9. Performance on Test Data

Loss is fine, but it doesn't show us the whole picture. And looking at the confusion matrix as numbers doesn't *show* us what's going on. So we're going to actually calculate the accuracy of your model from a visualized confusion matrix!

To do that, first we need to test your model on data that it hasn't yet seen or trained on. Remember when we split the dataset into 70% for `train_ds` and 30% for `test_ds`? We're testing on the cleaned version of `test_ds` which we named `clean_test_ds`. Test data is also called *unseen data*.

**Beginning Testing**

Run your model on the test data with this cell below.


In [ ]:
test_loss, test_tp, test_tn, test_fp, test_fn = model.evaluate(clean_test_ds, steps = math.ceil(NUM_TEST_IMAGES/BATCH_SIZE))

Using the Seaborn library we imported, we're going to draw the confusion matrix for your model to assess its performance on the test data!

Running these 2 cells below will build the confusion matrix for your model's performance on `clean_test_ds`.

In [ ]:
def draw_confusion_matrix(tp, tn, fp, fn):
  cf_matrix = np.array([[tp, fp], [fn, tn]])
  group_names = ['True Pos','False Pos','False Neg','True Neg']
  group_counts = ["{0:0.0f}".format(value) for value in cf_matrix.flatten()]
  group_percentages = ["{0:.2%}".format(value) for value in cf_matrix.flatten()/np.sum(cf_matrix)]
  labels = [f"{v1}\n{v2}\n{v3}" for v1, v2, v3 in zip(group_names,group_counts,group_percentages)]
  labels = np.asarray(labels).reshape(2,2)
  sns.heatmap(cf_matrix, annot = labels, fmt = '', cmap = 'Blues', xticklabels = False, yticklabels = False)

In [ ]:
draw_confusion_matrix(test_tp, test_tn, test_fp, test_fn)

Now, we can actually calculate the accuracy from the confusion matrix. Remember,

**Accuracy = (Total Trues) / (Total Predictions)**

Run the cell below to see the accuracy of your model on the test data!

In [ ]:
accuracy = (test_tp + test_tn) / (test_tp + test_tn + test_fp + test_fn)

print("The accuracy of your model is %.7f, or about %d%%." % (accuracy, round(accuracy*100)))
if (accuracy >= 0.95):
  print("Great job! Your high-accuracy AI is about to save Yew Nork City in style.")
elif (accuracy > 0.9):
  print("Good job! Your model is going to do a great job of helping Yew Nork City.")
elif (accuracy > 0.8):
  print("Not bad! Your model is going to prove fairly useful to Yew Nork City.")
else:
  print("Good effort, but Yew Nork City needs at least 80% test accuracy in order to diagnose patients accurately.")

# 10. Good job!

Well, you did it. If your model did better than 80%, BigTh!nk's AI engineers really mean business. Yew Nork City is eternally indebted to your service to its ailing civilians!

If you couldn't quite manage 80%, no worries. You can always make a copy of this project and tweak even more things to optimize your model even more. Go ahead and disable the cell where you saved your model, right above **9. Performance on Test Data**, to make infinite tweaks privately without overwriting your group's model. 

If you found this easy, you can always go up and set `NUMBER_OF_EPOCHS` in **08. Training** to be less than 5 and tune your model to be accurate with less epochs!

You can also go through the specific data processing functions in detail, if you'd like. 

Here are some documentation links if you want to learn more about the libraries, functions, and tools you (knowingly or unknowingly) used in this project!

*   [Malaria Dataset](https://www.tensorflow.org/datasets/catalog/malaria)
*   [TensorFlow](https://www.tensorflow.org/guide)
*   [Keras](https://keras.io/api/)
*   [Seaborn](https://seaborn.pydata.org/)
*   [Numpy](https://numpy.org/)
*   [Matplotlib](https://matplotlib.org/)